In [11]:
import heapq

class PuzzleState:
    def __init__(self, board, parent=None, move=0, cost=0, heuristic=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(tuple(self.board))

    def get_neighbors(self):
        neighbors = []
        zero_index = self.board.index(0)
        row, col = divmod(zero_index, 3)

        moves = {
            'Up': (row - 1, col),
            'Down': (row + 1, col),
            'Left': (row, col - 1),
            'Right': (row, col + 1)
        }

        for direction, (new_row, new_col) in moves.items():
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_zero_index = new_row * 3 + new_col
                new_board = self.board[:]
                new_board[zero_index], new_board[new_zero_index] = new_board[new_zero_index], new_board[zero_index]
                neighbors.append(PuzzleState(new_board, self, direction, self.cost + 1))
        
        return neighbors

    def get_path(self):
        path = []
        state = self
        while state:
            path.append(state)
            state = state.parent
        return path[::-1]

def misplaced_tiles_heuristic(state, goal):
    return sum(1 for i in range(9) if state.board[i] != 0 and state.board[i] != goal[i])

def manhattan_distance_heuristic(state, goal):
    distance = 0
    for i in range(1, 9):
        curr_pos = state.board.index(i)
        goal_pos = goal.index(i)
        curr_row, curr_col = divmod(curr_pos, 3)
        goal_row, goal_col = divmod(goal_pos, 3)
        distance += abs(curr_row - goal_row) + abs(curr_col - goal_col)
    return distance

import heapq

def a_star_search(start, goal, heuristic_func):
    open_list = []  # Priority queue to manage states to be explored
    heapq.heappush(open_list, (0, start))  # Push start state with initial cost 0
    closed_set = set()  # Set of visited states
    g_costs = {tuple(start.board): 0}  # Cost to reach each state

    while open_list:
        _, current_state = heapq.heappop(open_list)  # Get state with lowest f_cost

        if current_state.board == goal:
            return current_state.get_path()  # Return path if goal is reached

        closed_set.add(current_state)  # Mark current state as visited

        for neighbor in current_state.get_neighbors():
            if neighbor in closed_set:
                continue  # Skip already visited states

            tentative_g_cost = g_costs[tuple(current_state.board)] + 1  # Calculate cost to reach neighbor
            neighbor_board_tuple = tuple(neighbor.board)

            if tentative_g_cost < g_costs.get(neighbor_board_tuple, float('inf')):
                g_costs[neighbor_board_tuple] = tentative_g_cost
                neighbor.f_cost = tentative_g_cost + heuristic_func(neighbor, goal)
                
                heapq.heappush(open_list, (neighbor.f_cost, neighbor))  # Add neighbor to priority queue
    
    return None  # Return None if no path is found



def print_solution(path):
    for state in path:
        print(f"Move: {state.move}")
        print_board(state.board)
        print(f"Cost: {state.cost}, Heuristic: {state.heuristic}, Total: {state.cost + state.heuristic}")
        print()

def print_board(board):
    for i in range(3):
        print(board[3*i:3*i+3])
    print()

if __name__ == "__main__":
    start_board = [1, 2, 3, 4, 0, 5, 6, 7, 8]  # Start state
    goal_board = [1, 2, 3, 4, 5, 6, 7, 8, 0]   # Goal state

    start_state = PuzzleState(start_board)

    print("A* Search with Heuristic 1 (Misplaced Tiles):")
    path = a_star_search(start_state, goal_board, misplaced_tiles_heuristic)
    print_solution(path)

    print("A* Search with Heuristic 2 (Manhattan Distance):")
    path = a_star_search(start_state, goal_board, manhattan_distance_heuristic)
    print_solution(path)

    print("Best First Search with Heuristic 1 (Misplaced Tiles):")
    path = best_first_search(start_state, goal_board, misplaced_tiles_heuristic)
    print_solution(path)

    print("Best First Search with Heuristic 2 (Manhattan Distance):")
    path = best_first_search(start_state, goal_board, manhattan_distance_heuristic)
    print_solution(path)


A* Search with Heuristic 1 (Misplaced Tiles):
Move: 0
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Cost: 0, Heuristic: 0, Total: 0

Move: Down
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Cost: 1, Heuristic: 0, Total: 1

Move: Right
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Cost: 2, Heuristic: 0, Total: 2

A* Search with Heuristic 2 (Manhattan Distance):
Move: 0
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Cost: 0, Heuristic: 0, Total: 0

Move: Down
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Cost: 1, Heuristic: 0, Total: 1

Move: Right
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Cost: 2, Heuristic: 0, Total: 2

Best First Search with Heuristic 1 (Misplaced Tiles):
Move: 0
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Cost: 0, Heuristic: 0, Total: 0

Move: Down
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Cost: 1, Heuristic: 1, Total: 2

Move: Right
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Cost: 2, Heuristic: 0, Total: 2

Best First Search with Heuristic 2 (Manhattan Distance):
Move: 0
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Cost: 0, Heuristic: 0, Total: 0

Move: Down
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Cost: 1, He

In [3]:
class PuzzleState:
    def __init__(self, board, parent=None, move=0, cost=0, heuristic=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(tuple(self.board))

    def get_neighbors(self):
        neighbors = []
        zero_index = self.board.index(0)
        row, col = divmod(zero_index, 3)

        moves = {
            'Up': (row - 1, col),
            'Down': (row + 1, col),
            'Left': (row, col - 1),
            'Right': (row, col + 1)
        }

        for direction, (new_row, new_col) in moves.items():
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_zero_index = new_row * 3 + new_col
                new_board = self.board[:]
                new_board[zero_index], new_board[new_zero_index] = new_board[new_zero_index], new_board[zero_index]
                neighbors.append(PuzzleState(new_board, self, direction, self.cost + 1))
        
        return neighbors

    def get_path(self):
        path = []
        state = self
        while state:
            path.append(state)
            state = state.parent
        return path[::-1]

def misplaced_tiles_heuristic(state, goal):
    return sum(1 for i in range(9) if state.board[i] != 0 and state.board[i] != goal[i])

def manhattan_distance_heuristic(state, goal):
    distance = 0
    for i in range(1, 9):
        curr_pos = state.board.index(i)
        goal_pos = goal.index(i)
        curr_row, curr_col = divmod(curr_pos, 3)
        goal_row, goal_col = divmod(goal_pos, 3)
        distance += abs(curr_row - goal_row) + abs(curr_col - goal_col)
    return distance

def pop_min_cost_state(open_list):
    """Find and remove the state with the lowest cost + heuristic."""
    min_index = 0
    for i in range(1, len(open_list)):
        if open_list[i].cost + open_list[i].heuristic < open_list[min_index].cost + open_list[min_index].heuristic:
            min_index = i
    return open_list.pop(min_index)

def a_star_search(start, goal, heuristic_func):
    open_list = []
    open_list.append(start)
    closed_set = set()

    while open_list:
        current_state = pop_min_cost_state(open_list)

        if current_state.board == goal:
            return current_state.get_path()

        closed_set.add(current_state)

        for neighbor in current_state.get_neighbors():
            if neighbor in closed_set:
                continue

            neighbor.heuristic = heuristic_func(neighbor, goal)

            if neighbor not in open_list:
                open_list.append(neighbor)
    
    return None

def best_first_search(start, goal, heuristic_func):
    open_list = []
    open_list.append(start)
    closed_set = set()

    while open_list:
        current_state = pop_min_cost_state(open_list)

        if current_state.board == goal:
            return current_state.get_path()

        closed_set.add(current_state)

        for neighbor in current_state.get_neighbors():
            if neighbor in closed_set:
                continue

            neighbor.heuristic = heuristic_func(neighbor, goal)
            if neighbor not in open_list:
                open_list.append(neighbor)

    return None

def print_solution(path):
    for state in path:
        print(f"Move: {state.move}")
        print_board(state.board)
        print(f"Cost: {state.cost}, Heuristic: {state.heuristic}, Total: {state.cost + state.heuristic}")
        print()

def print_board(board):
    for i in range(3):
        print(board[3*i:3*i+3])
    print()

if __name__ == "__main__":
    start_board = [1, 2, 3, 4, 0, 5, 6, 7, 8]  # Start state
    goal_board = [1, 2, 3, 4, 5, 6, 7, 8, 0]   # Goal state

    start_state = PuzzleState(start_board)

    print("A* Search with Heuristic 1 (Misplaced Tiles):")
    path = a_star_search(start_state, goal_board, misplaced_tiles_heuristic)
    print_solution(path)

    print("A* Search with Heuristic 2 (Manhattan Distance):")
    path = a_star_search(start_state, goal_board, manhattan_distance_heuristic)
    print_solution(path)

    print("Best First Search with Heuristic 1 (Misplaced Tiles):")
    path = best_first_search(start_state, goal_board, misplaced_tiles_heuristic)
    print_solution(path)

    print("Best First Search with Heuristic 2 (Manhattan Distance):")
    path = best_first_search(start_state, goal_board, manhattan_distance_heuristic)
    print_solution(path)


A* Search with Heuristic 1 (Misplaced Tiles):
Move: 0
[1, 2, 3]
[4, 0, 5]
[6, 7, 8]

Cost: 0, Heuristic: 0, Total: 0

Move: Right
[1, 2, 3]
[4, 5, 0]
[6, 7, 8]

Cost: 1, Heuristic: 3, Total: 4

Move: Down
[1, 2, 3]
[4, 5, 8]
[6, 7, 0]

Cost: 2, Heuristic: 3, Total: 5

Move: Left
[1, 2, 3]
[4, 5, 8]
[6, 0, 7]

Cost: 3, Heuristic: 3, Total: 6

Move: Left
[1, 2, 3]
[4, 5, 8]
[0, 6, 7]

Cost: 4, Heuristic: 3, Total: 7

Move: Up
[1, 2, 3]
[0, 5, 8]
[4, 6, 7]

Cost: 5, Heuristic: 4, Total: 9

Move: Right
[1, 2, 3]
[5, 0, 8]
[4, 6, 7]

Cost: 6, Heuristic: 5, Total: 11

Move: Down
[1, 2, 3]
[5, 6, 8]
[4, 0, 7]

Cost: 7, Heuristic: 5, Total: 12

Move: Right
[1, 2, 3]
[5, 6, 8]
[4, 7, 0]

Cost: 8, Heuristic: 5, Total: 13

Move: Up
[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

Cost: 9, Heuristic: 5, Total: 14

Move: Left
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

Cost: 10, Heuristic: 4, Total: 14

Move: Left
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

Cost: 11, Heuristic: 3, Total: 14

Move: Down
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

Cost: 12